## Comprensión de lenguaje natural para sinopsis

### Introducción

En este *notebook* vamos a probar diversas técnicas de NLP para transformar una sinopsis de un libro en un vector que sirva de input para nuestro posterior modelo del sistema de recomendación. Trataremos de interpretar qué significado tienen esos vectores.

### Primeros pasos con spaCy

Usaremos, en principio, la librería NLP `spaCy`. En su caso, y como la gran mayoría de sinopsis de los libros de que disponemos son en inglés, utilizaremos el modelo (*pipeline*) `en_core_web_lg`. Hemos escogido la versión grande del modelo puesto que para la transformación de *tokens* de texto en vectores, cuenta con mucha más información precomputada para el cálculo de los mismos, además de ser más efectivo.

Mostramos un primer ejemplo en el que analizamos la sinopsis extraída de *GoodReads* del libro *The Dispossessed*, escrito por Ursula K. Le Guin.

In [1]:
import os
import spacy

# Lee la sinopsis de un libro
with open(os.path.join("summaries", "dispossessed.txt"), "r") as f:
    dispossessed_summary = f.read()

# Carga el modelo de spacy y tokeniza la sinopsis
nlp = spacy.load("en_core_web_lg")
dispossessed_tokens = nlp(dispossessed_summary)

# Imprime información sobre cada token y el potencial vector asociado
for token in dispossessed_tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

Shevek False 0.0 True
, True 64.72698 False
a True 112.98545 False
brilliant True 26.414904 False
physicist True 38.49606 False
, True 64.72698 False
decides True 40.19985 False
to True 125.107445 False
take True 67.411446 False
action True 63.787525 False
. True 59.90988 False
He True 127.80685 False
will True 67.574356 False
seek True 61.793026 False
answers True 41.63712 False
, True 64.72698 False
question True 45.032265 False
the True 72.329216 False
unquestionable True 28.41399 False
, True 64.72698 False
and True 60.75837 False
attempt True 47.164364 False
to True 125.107445 False
tear True 54.97283 False
down True 69.28324 False
the True 72.329216 False
walls True 58.260063 False
of True 120.9016 False
hatred True 40.90178 False
that True 57.417362 False
have True 61.392063 False
isolated True 40.373966 False
his True 95.241104 False
planet True 48.34115 False
of True 120.9016 False
anarchists True 33.042927 False
from True 58.585716 False
the True 72.329216 False
rest True 48.

Como vemos, la gran mayoría de *tokens* tienen un vector asociado, del que incluimos su norma L2 (la norma euclídea). Es llamativo cómo el primer *token*, referente a la palabra *Shevek*, no tiene información sobre un vector. Como indica el atributo `is_oov` (*out of vocabulary*), no es una palabra reconocible dentro del idioma inglés. Es lógico, pues se trata del protagonista de la novela, el cual procede de un planeta de otro sistema solar.

Podemos comparar también entre dos textos según su similitud. En este ejemplo, incluimos también la sinopsis del libro *The Moon is a harsh mistress*, de Robert A. Heinlein. Ambos libros comparten temas como sociedades utópicas, sistemas alternativos y reflexiones sobre la revolución, además de desarrollarse en lugares típicos de la ciencia ficción como pueden ser la luna o un planeta imaginario.

In [2]:
# Lee la sinopsis del segundo libro
with open(os.path.join("summaries", "harsh_mistress.txt"), "r") as f:
    harsh_mistress_summary = f.read()

# Tokeniza la sinopsis
harsh_mistress_tokens = nlp(harsh_mistress_summary)

# Calcula la similitud entre las sinopsis
print(dispossessed_tokens.similarity(harsh_mistress_tokens))

0.9282042345295354


Según la documentación de `spaCy`, para obtener la similitud se computa un vector de medias para cada uno de los textos a ser comparados. Así, el orden en que aparezcan las palabras dentro del texto no influye en el resultado. 

Además, dos textos que hablen de lo mismo pero que empleen palabras muy diferentes podrían tener un grado de similitud bajo. Por el contrario, si dos textos no necesariamente parecidos en contenido sí cuentan con una redacción similar, esto es, comparten un número importante de palabras, pueden obtener un grado de similitud elevado.

Vamos a comparar con una sinopsis de un libro de temática diferente para comprobar de nuevo esta métrica. Por ejemplo, probemos con *Wuthering Heights*, de Emily Brontë.

In [3]:
# Lee la sinopsis del tercer libro
with open(os.path.join("summaries", "wuthering.txt"), "r") as f:
    wuthering_summary = f.read()

# Tokeniza la sinopsis
wuthering_tokens = nlp(wuthering_summary)

# Calcula la similitud entre las sinopsis
print(dispossessed_tokens.similarity(wuthering_tokens))
print(harsh_mistress_tokens.similarity(wuthering_tokens))

0.8808053112027434
0.9473500192848667


La similitud sigue siendo bastante alta a pesar de lo diferentes que son los libros en cuanto a su contenido. 

Por tanto, podemos deducir que no sólo nos hace falta una representación vectorial "plana" de aquellas palabras que aparecen en una sinopsis de un libro en particular, sino que tendremos que aplicar técnicas algo más sofisticadas para dotar de significado a los textos y convertirlos en vectores que se adecuen más al contenido de estos. Esto es, actuar sobre la semántica de frases y textos cortos como un todo en el que el contexto y la semántica son tenidos en cuenta.

Probemos con SBERT, estado del arte para transformación de oraciones en vectores de alta dimensión. Probaremos con varios de los modelos ya entrenados disponibles en la librería `sentence-transformers`. 

Con esto, podremos calcular la similitud entre dos textos mediante la similitud coseno, tratando a cada uno de nuestros textos como vectores de un espacio euclídeo de dimensión elevada (el coseno del ángulo entre estos vectores nos dará una idea de la semejanza según su dirección y sentido).

In [7]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Cargamos el modelo BERT pre-entrenado
bert_model = SentenceTransformer('all-MiniLM-L12-v2')

# Codificamos las sinopsis en vectores (embeddings en un único tensor)
book_summaries = [dispossessed_summary, harsh_mistress_summary, wuthering_summary]
encoded_book_summaries = bert_model.encode(book_summaries, convert_to_tensor=True)

def pretty_print_embeddings(embeddings, labels):
    """
    Imprime los embeddings de cada libro
    """
    for i, embedding in enumerate(np.array(embeddings).tolist()):
        print("Libro: {}".format(labels[i]))
        print("Tamaño del embedding: {}".format(len(embedding)))
        embedding_snippet = ", ".join((str(x) for x in embedding[:3]))
        print("Embedding: [{}, ...]\n".format(embedding_snippet))

# Representación de los embeddings
titles = ['The Dispossessed', 'The Moon is a Harsh Mistress', 'Wuthering Heights']
pretty_print_embeddings(encoded_book_summaries, titles)

# Calculamos la similitud entre las sinopsis (matriz de cosine similarity)
print(util.cos_sim(encoded_book_summaries, encoded_book_summaries))

0.7992377877235413
0.4220314919948578
0.4273483455181122


Para una representación más visual, podemos considerar un gráfico de calor que muestre la matriz de similitudes. Se usa la librería `seaborn`.

In [ ]:
import seaborn as sns

def plot_similarity(features, labels, rotation, similarity):
    """
    Representa la similitud entre embeddings con un mapa de calor
    """
    corr = similarity(features, features)
    print(corr)
    sns.set(font_scale=1.2)
    g = sns.heatmap(
        corr,
        xticklabels=labels,
        yticklabels=labels,
        vmin=0,
        vmax=1,
        cmap="YlOrRd"
    )
    g.set_xticklabels(labels, rotation=rotation)
    g.set_title("Similitud semántica entre sinopsis")

plot_similarity(encoded_book_summaries, titles, 90, util.cos_sim)

In [ ]:
"""
Versión del anterior menos potente, pero mucho más rápida
Embeddings de 384 dimensiones y normalizados
"""
bert_model = SentenceTransformer('all-MiniLM-L6-v2') 

# Codificamos las sinopsis en vectores (embeddings en un único tensor)
encoded_book_summaries = bert_model.encode(book_summaries, convert_to_tensor=True)

# Representación de los embeddings
pretty_print_embeddings(encoded_book_summaries, titles)

# Se usa producto escalar al estar los embeddings normalizados
plot_similarity(encoded_book_summaries, titles, 90, util.dot_score)

In [ ]:
"""
Segundo modelo más potente
768 dimensiones, embeddings normalizados
"""
bert_model = SentenceTransformer('all-distilroberta-v1')

In [ ]:
"""
Modelo más potente
768 dimensiones, embeddings normalizados
"""
bert_model = SentenceTransformer('all-mpnet-base-v2')

Otro modelo actual para comparación de texto es USE (*Universal Sentence Encoder*), de Google.

### TF-IDF (Term Frequency-Inverse Document Frequency)

### Word Embeddings (Word2Vec, GloVe, BERT)

### Sentiment analysis